In [2]:
import cv2
import torch
import numpy as np
from models import yolo,common
from utils.datasets import LoadImages
from utils.general import non_max_suppression
from utils.plots import plot_one_box

In [3]:
model = yolo.Model(cfg = "models/yolov5s.yaml")

In [4]:
chkpt = torch.load("yolov5s.pt")
state = chkpt["model"].float().state_dict()
model.load_state_dict(state)

<All keys matched successfully>

In [5]:
model.info(verbose = False)

In [6]:
img_path = ["data/images/zidane.jpg","data/images/bus.jpg"]
img_path_2 = ["https://ultralytics.com/images/zidane.jpg"]

In [7]:
dataset = LoadImages("data/images")

model.eval()
for path, img, im0s, vid_cap in dataset:
    with torch.no_grad():
        img = torch.from_numpy(img)
        img = img.float()          # uint8 to fp16/32
        img /= 255.0                                       # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        print("the shape of input",img.shape)   
        result = model(img)[0]
        result = non_max_suppression(result)[0]
        print()
        #print(result)
        print("The shape of returned tensor ",result.shape)

image 1/2 /home/ankit/Documents/ProductRecognition/CODE/yolov5/data/images/bus.jpg: the shape of input torch.Size([1, 3, 640, 480])

The shape of returned tensor  torch.Size([6, 6])
image 2/2 /home/ankit/Documents/ProductRecognition/CODE/yolov5/data/images/zidane.jpg: the shape of input torch.Size([1, 3, 384, 640])

The shape of returned tensor  torch.Size([4, 6])


In [12]:
img1 = cv2.imread(img_path[0])
img2 = cv2.imread(img_path[1])
img1 = cv2.resize(img1,(640,640))
img2 = cv2.resize(img2,(640,640))
imgs = np.stack([img1,img2])
print(imgs.shape)
imgs = torch.from_numpy(imgs)
imgs = imgs.permute(0,3,2,1)
print(imgs.shape)
imgs = imgs.float()
result = model(imgs)[0]
print(result[0].size(),result[1].size())
result[0] = non_max_suppression(result[0])[0]
result[1] = non_max_suppression(result[1])[0]
print(result[0].shape,result[1].shape)

(2, 640, 640, 3)
torch.Size([2, 3, 640, 640])
torch.Size([25200, 85]) torch.Size([25200, 85])


IndexError: tuple index out of range

In [11]:
for i, det in enumerate(result):  # detections per image
    
    im0 = cv2.imread(img_path[i],-1)
#     if webcam:  # batch_size >= 1
#         p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
#     else:
#         p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

#     p = Path(p)  # to Path
#     save_path = str(save_dir / p.name)  # img.jpg
#     txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
#     s += '%gx%g ' % img.shape[2:]  # print string
#     gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
#     imc = im0.copy() if opt.save_crop else im0  # for opt.save_crop
    if len(det):
        # Rescale boxes from img_size to im0 size
        #det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

        # Print results
#         for c in det[:, -1].unique():
#             n = (det[:, -1] == c).sum()  # detections per class
#             s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

        # Write results
        for *xyxy, conf, cls in reversed(det):
#             if save_txt:  # Write to file
#                 xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
#                 line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
#                 with open(txt_path + '.txt', 'a') as f:
#                     f.write(('%g ' * len(line)).rstrip() % line + '\n')

#             if save_img or opt.save_crop or view_img:  # Add bbox to image
            c = int(cls)  # integer class
            label = f'{c} {conf:.2f}'
            plot_one_box(xyxy, im0, label = label)
                

    # Stream results
    cv2.imshow("windoW", im0)
    cv2.waitKey(2000)  # 1 millisecond
    cv2.destroyAllWindows()

TypeError: iteration over a 0-d tensor